In [19]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer  
from sklearn import cross_validation
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import TruncatedSVD

In [20]:
df=pd.read_csv('train_modified.csv')
normal=df[df['flag']=='n']
webshell=df[df['flag']=='w']
n_query=[]
w_query=[]
for item in normal['clean_action']:
	n_query.append(str(item))
for item in webshell['clean_action']:
	w_query.append(str(item))
queries=n_query+w_query
n_y = [0 for i in range(0,len(n_query))]
w_y = [1 for i in range(0,len(w_query))]
y = n_y + w_y

In [21]:
def get_ngrams(query):
    ngrams = []
    text = nltk.word_tokenize(str(query))
    for w in text:
    	w=w.lower().lstrip(' ').rstrip(' ').replace("u'",' ')
    	if w[0]=="'":
    		w=w[1:-1]
    	w=w.lstrip(' ').rstrip(' ')
    	ngrams.append(w)
    return ngrams

In [25]:
#selection Tfidf
vectorizer = TfidfVectorizer(tokenizer=get_ngrams)
X = vectorizer.fit_transform(queries)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=8)
# print X_train.shape

In [5]:
#selection Hashing
vectorizer = HashingVectorizer(tokenizer=get_ngrams,n_features = 500)  
X= vectorizer.fit_transform(queries)  
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=4)
print X.shape
print X_train.shape

(739, 500)
(591, 500)


In [ ]:
# selection CountVectorizer
vectorizer = CountVectorizer(tokenizer=get_ngrams,n_features=500)
X = vectorizer.fit_transform(queries)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=4)


In [26]:
#SVD降维
svd=TruncatedSVD(n_components=1500)
svd.fit(X)
X=svd.transform(X)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=4)


In [11]:
#Model KNN
k_range=range(1,30)
cv_scores = []
for k in k_range:
    clf = KNeighborsClassifier(k) 
    scores =cross_validation .cross_val_score(clf, X_train, y_train, cv=10, scoring='f1')
    cv_scores.append(scores.mean()) 
print cv_scores


[0.82377423492918478, 0.79336731265136839, 0.80845616839460899, 0.80844875154085682, 0.69711767550651005, 0.69893639953888731, 0.69753041201056809, 0.69034330564681867, 0.6928446560321293, 0.66549269207146644, 0.65943054456666195, 0.64191939434247691, 0.64149881898287708, 0.63164395301041254, 0.63049084249084242, 0.6301805383762662, 0.62443933395199758, 0.61595534779359284, 0.61162456796649989, 0.61348287226382858, 0.61517943436123357, 0.6164354730085766, 0.62255501145518344, 0.61657459001831594, 0.6192849424519723, 0.61397561872248052, 0.61209146176985274, 0.6036542026504701, 0.60238570370755262]


In [31]:
#Model Decision Tree
clf=tree.DecisionTreeClassifier(splitter='random',max_features=700,max_depth=12,min_samples_leaf=1)
clf.fit(X_train,y_train)

# scores =cross_validation .cross_val_score(clf, X_train, y_train, cv=10, scoring='f1')
#clf.feature_importances_
# print scores.mean()

0.827530146261


[ 0.51851852  0.89473684  0.78787879  0.82352941  0.57142857  0.88888889
  0.70967742  0.70967742  0.77419355  0.70967742]
